# MNIST 복습

In [1]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
import matplotlib.pyplot as plt

# MNIST데이터 로드
mnist = keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# 모델에 맞게 데이터 가공
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
x_train_reshaped = x_train_norm.reshape(-1, x_train_norm.shape[1]*x_train_norm.shape[2])
x_test_reshaped = x_test_norm.reshape(-1, x_test_norm.shape[1]*x_test_norm.shape[2])

# 딥러닝 모델 구성 -2 Layer Perceptron
model = keras.models.Sequential()
# 입력층 d=784, 은닉층레이어 H=50
model.add(keras.layers.Dense(50, activation='sigmoid', input_shape=(784,)))
# 출력층 레이어 K=10
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

# 모델 구성과 학습
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train_reshaped, y_train, epochs=10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 4s 69us/sample - loss: 0.4998 - accuracy: 0.8811
Epoch 2/10
60000/60000 [==============================] - 4s 63us/sample - loss: 0.2282 - accuracy: 0.9360
Epoch 3/10
60000/60000 [==============================] - 4s 63us/sample - loss: 0.1785 - accuracy: 0.9487
Epoch 4/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.1486 - accuracy: 0.9572
Epoch 5/10
60000/60000 [==============================] - 5s 80

In [2]:
# 모델 테스트 결과
# verbose2 :epoch당 1줄의 로그
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print('test_loss:{}'.format(test_loss))
print('test_accuracy:{}'.format(test_accuracy))  #format의 다른활용 확인

10000/10000 - 0s - loss: 0.1011 - accuracy: 0.9681
test_loss:0.10107238476648926
test_accuracy:0.9681000113487244


# 신경망 구성

* Parameters/Weights

In [3]:
# 입력층 데이터의 모양(shpae)
print(x_train_reshaped.shape)

# 테스트를 위해 x_train_reshaped의 앞 5개의 데이터를 가져온다.
X = x_train_reshaped[:5]
print(X.shape)

(60000, 784)
(5, 784)


In [4]:
weight_init_std = 0.1
input_size = 784
hidden_size = 50

# 인접 레이어간 관계를 나타내는 파라미터 W를 생성하고 random 초기화
W1 = weight_init_std * np.random.randn(input_size, hidden_size)
# 바이어스 파라미터 b를 생성하고 zero로 초기화
b1 = np.zeros(hidden_size)

a1 = np.dot(X,W1)+b1 # 은닉층 출력

print(W1.shape)
print(b1.shape)
print(a1.shape)

(784, 50)
(50,)
(5, 50)


In [5]:
# 첫번째 데이터의 은닉층 출력 확인, 50dim?
a1[0]

array([ 0.25037035,  0.99607237, -0.88241222,  0.39543646,  1.01191845,
       -0.13792064,  0.195563  , -0.92404164, -0.34318827,  0.29141884,
        0.98159909,  1.67089526,  0.35913317, -0.98661716,  0.40519472,
       -0.63136321, -0.16141909, -0.1717446 , -1.33916511,  1.57111586,
        0.5040565 , -0.53763192,  0.07662242,  0.46082249,  1.05778741,
       -0.37550594,  1.37737741,  1.46993102, -0.48363333,  0.04904029,
        0.06277108,  0.92732853,  1.44976247, -1.02325466, -0.21709279,
       -0.33043888, -0.33923657, -1.45466922,  0.69847378,  1.57419142,
        0.25199016,  0.90689242, -1.1302023 , -0.04163388, -0.31171373,
       -1.22905772,  0.54804233, -1.06952156,  0.95197816, -0.93668602])

* 활성화 함수

In [ ]:
# 마크다운 이미지 적용시 git/image폴더에 이동시킨후 상대경로 사용 (./image~~)

![img](./image/sigmoid.png)

![img](./image/relu.png)

In [6]:
# sigmoid함수 구현
def sigmoid(x):
    return 1/(1+np.exp(-x))

z1 = sigmoid(a1)
print(z1[0]) #sigmoid : 0~1 사이

[0.56226765 0.73028566 0.29267816 0.59759073 0.73339542 0.46557439
 0.54873552 0.2841351  0.41503522 0.57234345 0.72742539 0.84169515
 0.58883058 0.27158077 0.59993511 0.3472015  0.45973262 0.45716908
 0.20764739 0.82794263 0.62341215 0.36873863 0.51914624 0.61320927
 0.74226749 0.40721127 0.79856947 0.8130469  0.38139454 0.51225762
 0.51568762 0.71653299 0.80996188 0.26439392 0.44593896 0.41813384
 0.41599494 0.18928401 0.6678493  0.82838031 0.56266629 0.71236384
 0.24412377 0.48959303 0.42269649 0.22634639 0.63368128 0.25549408
 0.72151283 0.28157024]


In [7]:
# 단일 레이어 구현 함수
def affine_layer_forward(X, W, b):
    y = np.dot(X, W)+b
    cache = (X, W ,b)
    return y, cache

print('go~')

go~


In [8]:
input_size = 784
hidden_size = 50
output_size = 10

W1 = weight_init_std * np.random.randn(input_size, hidden_size)
b1 = np.zeros(hidden_size)
W2 = weight_init_std * np.random.randn(hidden_size, output_size)
b2 = np.zeros(output_size)

a1, cache1 = affine_layer_forward(X, W1, b1)
z1 = sigmoid(a1)
# z1이 다시 두번째 레이어의 입력이 됩니다
a2, cache2 = affine_layer_forward(z1, W2, b2)

print(a2[0]) # 최종출력이 output_size만큼의 벡터가 되었습니다.


[ 0.30693519  0.64105596  0.62853352 -0.18883395 -0.19498565  0.29508387
 -0.585775    0.27363648 -0.69299099 -0.07941346]


In [9]:
# a2에 softmax함수 적용 >> 입력X가 10가지 숫자중 하나일 확률
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T
    
    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))
        

In [10]:
y_hat = softmax(a2)
y_hat[0]  # 10개의 숫자 중 하나일 확률이 되었습니다.

array([0.11900563, 0.16621656, 0.1641481 , 0.07248659, 0.07204205,
       0.11760358, 0.04873808, 0.11510814, 0.04378295, 0.08086832])

* Loss function

In [11]:
# 정답 라벨을 One-hot 인코딩하는 함수
def _change_ont_hot_label(X, num_category):
    T = np.zeros((X.size, num_category))
    for idx, row in enumerate(T):
        row[X[idx]] = 1
    
    return T

Y_digit = y_train[:5]
t = _change_ont_hot_label(Y_digit, 10)
t # 정답 라벨의 one-hot 인코딩

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [12]:
# 모델의 최종출력 / 정답라벨의 one-hot인코딩 유사?
print(y_hat[0]) # 4번째?
print(t[0])

[0.11900563 0.16621656 0.1641481  0.07248659 0.07204205 0.11760358
 0.04873808 0.11510814 0.04378295 0.08086832]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [13]:
sum(y_hat[0])

1.0000000000000002

In [14]:
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1,t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t]))/batch_size

Loss = cross_entropy_error(y_hat, t)
Loss

2.303327310160455

# 경사하강법